# Reducing science images


In [ ]:
from pathlib import Path

from matplotlib import pyplot as plt

from astropy import units as u
import ccdproc as ccdp

from convenience_functions import show_image

## Overview

[ccdproc](ccdproc.readthedocs.io) provides a couple of ways to approach calibration of the science images:

+ Perform each of the individual steps manually using `subtract_bias`, `subtract_dark`, and `flat_correct`.
+ Use the [`ccd_process` function]() to perform all of the reduction steps. 

This notebook will do each of those in separate sections below.

## Example 1

## Example 2

In [ ]:
reduced_path = Path('example2-reduced')
ifc_reduced = ccdp.ImageFileCollection('example2-reduced')
ifc_raw = ccdp.ImageFileCollection('example-thermo-electric')

Grab the light images...

In [ ]:
lights = ifc_raw.summary[ifc_raw.summary['imagetyp'] == 'LIGHT']
lights['date-obs', 'file', 'object', 'filter', 'exposure']

Grab some calibration frames

In [ ]:
calibs = {}
for im_type in ['bias', 'dark', 'flat']:
    calibs[im_type] = [ccd for ccd in ifc_reduced.ccds(imagetyp=im_type, combined=True)]

Let's reduce them all...

In [ ]:
all_reds = []
light_ccds = []
for light, file_name in ifc_raw.ccds(imagetyp='light', return_fname=True):
    save_name, _ = file_name.split('.gz')
    light_ccds.append(light)
    reduced = ccdp.trim_image(light[:, :4096])
    reduced = ccdp.subtract_bias(reduced, calibs['bias'][0])
    reduced = ccdp.subtract_dark(reduced, calibs['dark'][0], exposure_time='exptime', exposure_unit=u.second)
    reduced = ccdp.flat_correct(reduced, calibs['flat'][0])
    all_reds.append(reduced)
    reduced.write(reduced_path / save_name)

In [ ]:
which_light = 0
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

axes[0].set_title('Uncalibrated image')
show_image(light_ccds[which_light], cmap='gray', ax=axes[0], fig=fig, percl=90)

axes[1].set_title('Calibrated image')
show_image(all_reds[which_light].data, cmap='gray', ax=axes[1], fig=fig, percl=90)

In [ ]:
which_light = 1
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

axes[0].set_title('Uncalibrated image')
show_image(light_ccds[which_light], cmap='gray', ax=axes[0], fig=fig, percl=90)

axes[1].set_title('Calibrated image')
show_image(all_reds[which_light].data, cmap='gray', ax=axes[1], fig=fig, percl=90)